In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def get_opgg_match_stats(summoner_name, tagline='NA1', region='na'):
    """Scrape last 10 matches' win/loss, gold diff, and dragons from OP.GG"""
    
    # Set up Selenium with options
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Chrome(options=options)
    
    # Construct correct URL format
    url = f"https://www.op.gg/summoners/{region}/{summoner_name}-{tagline}"
    
    try:
        # Load page
        driver.get(url)
        print(f"Loaded URL: {url}")  # Debug print
        
        # First wait for the page to load (check for summoner name element)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h1.summoner-name"))
        )
        print("Summoner name found")  # Debug print
        
        # Wait specifically for match history section
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1g3kmsj.e1iiyghw6"))
        )
        print("Match history section found")  # Debug print
        
        # Scroll to load more matches
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Parse with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Prepare data list
        matches = []
        
        # Extract data from each match (last 10)
        match_items = soup.select('div.css-1g3kmsj.e1iiyghw6 > div')
        print(f"Found {len(match_items)} match items")  # Debug print
        
        for match in match_items[:10]:
            try:
                # Win/Loss
                result = match.select_one('div.result').get_text(strip=True) if match.select_one('div.result') else None
                
                # Gold difference
                gold_diff = match.select_one('div.stats div.gold').get_text(strip=True) if match.select_one('div.stats div.gold') else None
                
                # Dragons taken
                dragons = match.select_one('div.stats div.dragon').get_text(strip=True) if match.select_one('div.stats div.dragon') else None
                
                matches.append({
                    'result': result,
                    'gold_diff': gold_diff,
                    'dragons': dragons
                })
            except Exception as e:
                print(f"Error processing match: {str(e)}")
                continue
        
        # Create DataFrame
        df = pd.DataFrame(matches)
        
        # Clean data if we got results
        if not df.empty:
            if 'gold_diff' in df.columns:
                df['gold_diff'] = df['gold_diff'].str.extract(r'([\d.]+)k')[0].astype(float) * 1000
            if 'dragons' in df.columns:
                df['dragons'] = pd.to_numeric(df['dragons'].str.extract(r'(\d+)')[0], errors='coerce').fillna(0).astype(int)
        
        return df
    
    except Exception as e:
        print(f"Error during scraping: {str(e)}")
        return pd.DataFrame()  # Return empty DataFrame on error
    finally:
        driver.quit()

# Example usage
if __name__ == "__main__":
    df = get_opgg_match_stats("X8Scorpio8X", "NA1", "na")
    if not df.empty:
        print("\nMatch History Stats:")
        print(df)
    else:
        print("Failed to retrieve match data")

Loaded URL: https://www.op.gg/summoners/na/X8Scorpio8X-NA1
Error during scraping: Message: 
Stacktrace:
0   chromedriver                        0x00000001034eea54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x00000001034e6cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x0000000103032864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000103079410 cxxbridge1$string$len + 382736
4   chromedriver                        0x00000001030ba480 cxxbridge1$string$len + 649088
5   chromedriver                        0x000000010306d7ec cxxbridge1$string$len + 334572
6   chromedriver                        0x00000001034b3ccc cxxbridge1$str$ptr + 2562928
7   chromedriver                        0x00000001034b6f98 cxxbridge1$str$ptr + 2575932
8   chromedriver                        0x00000001034942c4 cxxbridge1$str$ptr + 2433384
9   chromedriver                        0x00000001034b7810 cxxbridge1$str$ptr + 2578100
10  chrom

In [5]:
summoner_name = "hello"
print(f"https://www.op.gg/summoners/na/{summoner_name.replace(' ', '%20')}")

https://www.op.gg/summoners/na/hello
